In [23]:
import pandas as pd

## Model Training

In [24]:
df = pd.read_csv('./data/gemstone.csv')
df.head()

id  carat        cut color clarity  depth  table     x     y     z  price
0   0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55  13619
1   1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05  13387
2   2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50   2772
3   3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71    666
4   4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77  14453

In [25]:
df=df.drop(labels=['id'],axis=1)

In [26]:
## Independent and Dependent Features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [27]:
Y

price
0       13619
1       13387
2        2772
3         666
4       14453
...       ...
193568   1130
193569   2874
193570   3036
193571    681
193572   2258

[193573 rows x 1 columns]

In [28]:

# Defining which columns should be ordinal-encoded and which should be scaled.
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [29]:
# DefinING the custom ranking for each ordinal variable.
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [30]:
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import StandardScaler # Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [31]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [32]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [33]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [34]:
X_train.head()

num_pipeline__carat  num_pipeline__depth  num_pipeline__table  \
0            -0.975439            -0.849607            -0.121531   
1             0.235195             1.833637            -0.121531   
2             0.494617             0.815855             0.399800   
3            -1.018676             0.260701             0.921131   
4            -0.953821            -0.664555            -0.642862   

   num_pipeline__x  num_pipeline__y  num_pipeline__z  cat_pipeline__cut  \
0        -1.042757        -1.080970        -1.123150           0.874076   
1         0.318447         0.279859         0.485354          -2.144558   
2         0.570855         0.606458         0.673737          -0.132136   
3        -1.214034        -1.244270        -1.195605          -0.132136   
4        -1.069801        -1.044681        -1.094168           0.874076   

   cat_pipeline__color  cat_pipeline__clarity  
0             1.528722               1.352731  
1            -0.935071              -0.646786  
2             0.296826               0.686225  
3             0.296826               0.019720  
4             2.144670               1.352731

In [35]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [36]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook.
On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.

In [37]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [38]:
regression.intercept_

array([3970.76628955])

In [39]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [40]:
## Training Multiple Models
# Model Evaluation

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    # Making Predictions

    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9047094344004
MAE: 674.0255115796832
R2 score 93.68908248567512


Lasso
Model Training Performance
RMSE: 1013.8784226767013
MAE: 675.0716923362158
R2 score 93.68940971841704


Ridge
Model Training Performance
RMSE: 1013.9059272771643
MAE: 674.0555800798212
R2 score 93.68906732505938


Elasticnet
Model Training Performance
RMSE: 1533.4162456064048
MAE: 1060.7368759154729
R2 score 85.56494831165182




In [41]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']